In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import random



In [3]:
TRAIN_DIR = "C:/Users/Pongo/Documents/Data/Personal/Competition/Compfest/emotion-detector-jupyter/data/train"
TEST_DIR = "C:/Users/Pongo/Documents/Data/Personal/Competition/Compfest/emotion-detector-jupyter/data/test"

In [4]:
def load_data(data_dir, image_size=(48, 48)):
    images = []
    labels = []
    training_data = []
    for emotion_dir in os.listdir(data_dir):
        label = os.listdir(data_dir).index(emotion_dir)  # Assuming folder names are labels (0-6)
        emotion_dir_path = os.path.join(data_dir, emotion_dir)
        for img_name in os.listdir(emotion_dir_path):
            img_path = os.path.join(emotion_dir_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, image_size)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = img_to_array(img) / 255.0 
            training_data.append([img, label])

    random.shuffle(training_data)
    for features, label in training_data:
        images.append(features)
        labels.append(label)

    return np.array(images), np.array(labels)

X_train, y_train = load_data(TRAIN_DIR)

In [5]:
from tensorflow.keras.utils import to_categorical

num_classes = 7
y_train = to_categorical(y_train, num_classes=num_classes)

In [6]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Activation

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(num_classes, activation='softmax')(x)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(64)(x)
x = Activation("relu")(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


C:\Users\Pongo\AppData\Local\Temp\ipykernel_24208\1236682268.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 24, 24,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 24, 24,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 24, 24,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 24, 24,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 24, 24,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 24, 24,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 24, 24,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 24, 24,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 24, 24,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 25, 25,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 12, 12,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 12, 12,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 12, 12,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 12, 12,    │      2,304 │ block_1_depthwis

 Total params: 7,223,767 (27.56 MB)

 Trainable params: 2,396,551 (9.14 MB)

 Non-trainable params: 34,112 (133.25 KB)

 Optimizer params: 4,793,104 (18.28 MB)

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 104s 105ms/step - accuracy: 0.3542 - loss: 1.6736 - val_accuracy: 0.3809 - val_loss: 2.1507
Epoch 2/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 75s 105ms/step - accuracy: 0.4810 - loss: 1.3692 - val_accuracy: 0.3598 - val_loss: 3.0337
Epoch 3/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 77s 107ms/step - accuracy: 0.5014 - loss: 1.2984 - val_accuracy: 0.4218 - val_loss: 2.5341
Epoch 4/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 80s 111ms/step - accuracy: 0.5226 - loss: 1.2688 - val_accuracy: 0.2849 - val_loss: 3.0741
Epoch 5/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 76s 106ms/step - accuracy: 0.5053 - loss: 1.3012 - val_accuracy: 0.3048 - val_loss: 2.1096
Epoch 6/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 79s 110ms/step - accuracy: 0.4493 - loss: 1.4301 - val_accuracy: 0.3520 - val_loss: 2.5334
Epoch 7/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 78s 109ms/step - accuracy: 0.4875 - loss: 1.3295 - val_accuracy: 0.3149 - val_loss: 2.0102
Epoch 8/100
718/718 ━━━━━━━━━━━━━━━━━━━━ 78s 108ms/step - accuracy: 0.5136 

In [10]:
model.save("model.h5")